In [1]:
# Hücre 1: Kütüphaneleri import edin
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.io
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("Kütüphaneler başarıyla yüklendi!")

Kütüphaneler başarıyla yüklendi!


In [2]:
import scipy.io

file_path = "../data/raw/B0005.mat"
mat_data = scipy.io.loadmat(file_path)

print("Anahtarlar:", mat_data.keys())

# Ana veri anahtarını bul
for key in mat_data.keys():
    if not key.startswith("__"):
        print("\n=== Anahtar:", key, "===")
        print("Tip:", type(mat_data[key]))
        print("Shape:", getattr(mat_data[key], "shape", None))

        # Eğer structured array ise alan isimlerini göster
        if hasattr(mat_data[key], "dtype") and mat_data[key].dtype.names:
            print("Alanlar:", mat_data[key].dtype.names)


Anahtarlar: dict_keys(['__header__', '__version__', '__globals__', 'B0005'])

=== Anahtar: B0005 ===
Tip: <class 'numpy.ndarray'>
Shape: (1, 1)
Alanlar: ('cycle',)


In [3]:
import scipy.io
import numpy as np
import pandas as pd

def safe_extract(array):
    while isinstance(array, np.ndarray) and array.size == 1:
        array = array[0]
    return array

# .mat dosyasını yükle
mat = scipy.io.loadmat("../data/raw/B0005.mat")
battery = mat["B0005"][0,0]

cycles = battery["cycle"][0]
print(f"Toplam çevrim: {len(cycles)}")

records = []

for i, cycle in enumerate(cycles[:3]):  # ilk 3 çevrim
    cycle_struct = safe_extract(cycle)
    cycle_type = safe_extract(cycle_struct["type"])
    ambient_temp = float(safe_extract(cycle_struct["ambient_temperature"]))
    data = safe_extract(cycle_struct["data"])

    voltage = safe_extract(data["Voltage_measured"]).flatten()
    current = safe_extract(data["Current_measured"]).flatten()
    temperature = safe_extract(data["Temperature_measured"]).flatten()
    time = safe_extract(data["Time"]).flatten()

    records.append({
        "cycle": i+1,
        "type": cycle_type,
        "ambient_temperature": ambient_temp,
        "voltage_mean": float(np.mean(voltage)),
        "current_mean": float(np.mean(current)),
        "temperature_mean": float(np.mean(temperature)),
        "time_max": float(np.max(time))
    })

df = pd.DataFrame(records)
print(df)



Toplam çevrim: 616
   cycle       type  ambient_temperature  voltage_mean  current_mean  \
0      1     charge                 24.0      4.187420      0.643455   
1      2  discharge                 24.0      3.529829     -1.818702   
2      3     charge                 24.0      4.058826      0.949043   

   temperature_mean   time_max  
0         25.324079   7597.875  
1         32.572328   3690.234  
2         26.635623  10516.000  


In [21]:
print(cycle_struct.dtype.names)


('type', 'ambient_temperature', 'time', 'data')


In [2]:
# Hücre 2: NASA veri setini yükleyin
def load_nasa_battery_data(file_path):
    """
    NASA MATLAB dosyasını yükler ve DataFrame'e çevirir
    """
    try:
        # MATLAB dosyasını yükle
        mat_data = scipy.io.loadmat(file_path)
        print(f"Dosya başarıyla yüklendi: {file_path}")
        
        # Dosya yapısını inceleyin
        print("Dosya anahtarları:", list(mat_data.keys()))
        
        # Batarya verisi genellikle bu anahtarlarda bulunur
        battery_keys = [key for key in mat_data.keys() if not key.startswith('__')]
        print("Batarya veri anahtarları:", battery_keys)
        
        return mat_data
    except Exception as e:
        print(f"Hata: {e}")
        return None

# Veri dosyasının yolunu belirtin (kendi dosya yolunuzu yazın)
data_path = Path("../data/raw/B0005.mat")  

# Eğer dosya varsa yükle
if data_path.exists():
    battery_data = load_nasa_battery_data(data_path)
else:
    print(f"Dosya bulunamadı: {data_path}")
    print("Lütfen NASA veri setini data/raw/ klasörüne koyduğunuzdan emin olun")

Dosya başarıyla yüklendi: ..\data\raw\B0005.mat
Dosya anahtarları: ['__header__', '__version__', '__globals__', 'B0005']
Batarya veri anahtarları: ['B0005']


In [10]:
def explore_battery_structure(mat_data):
    """
    Batarya verisi yapısını güvenli bir şekilde detaylı inceler
    """
    if mat_data is None:
        return
    
    # Ana veri anahtarını bul
    main_key = None
    for key in mat_data.keys():
        if not key.startswith('__') and isinstance(mat_data[key], np.ndarray):
            main_key = key
            break
    
    if main_key:
        print(f"Ana veri anahtarı: {main_key}")
        battery_struct = mat_data[main_key]
        print(f"Veri tipi: {type(battery_struct)}")
        print(f"Veri şekli: {battery_struct.shape}")
        
        # Eğer structured array ise alanları listele
        if hasattr(battery_struct, 'dtype') and battery_struct.dtype.names:
            print("Veri alanları:", battery_struct.dtype.names)
            
            # Her alanın içeriğini güvenli bir şekilde al
            for field_name in battery_struct.dtype.names:
                field_data = battery_struct[field_name]
                
                # Eğer object array ise 0. elemanı al
                while isinstance(field_data, np.ndarray) and field_data.size == 1:
                    field_data = field_data[0]
                
                print(f"\n{field_name}:")
                print(f"  - Tip: {type(field_data)}")
                if hasattr(field_data, 'shape'):
                    print(f"  - Şekil: {field_data.shape}")
    
    return main_key

# Veri yapısını keşfedin
if 'battery_data' in locals():
    main_key = explore_battery_structure(battery_data)


Ana veri anahtarı: B0005
Veri tipi: <class 'numpy.ndarray'>
Veri şekli: (1, 1)
Veri alanları: ('cycle',)

cycle:
  - Tip: <class 'numpy.ndarray'>
  - Şekil: (1, 616)


In [5]:
def safe_extract(array):
    """
    Nested (iç içe) numpy array'leri açar ve son elemanı döndürür
    """
    while isinstance(array, np.ndarray) and array.size == 1:
        array = array[0]
    return array


def explore_battery_structure(mat_data):
    if mat_data is None:
        return
    
    main_key = None
    for key in mat_data.keys():
        if not key.startswith('__') and isinstance(mat_data[key], np.ndarray):
            main_key = key
            break
    
    if main_key:
        battery_struct = mat_data[main_key]
        
        if hasattr(battery_struct, 'dtype') and battery_struct.dtype.names:
            for field_name in battery_struct.dtype.names:
                field_data = battery_struct[field_name]
                field_data = safe_extract(field_data)  # Burada kullanıyoruz
                
                print(f"{field_name} tip: {type(field_data)}, şekil: {getattr(field_data,'shape', None)}")
    
    return main_key

# Kullanım
if 'battery_data' in locals():
    main_key = explore_battery_structure(battery_data)


In [14]:
def explore_battery_structure(mat_data):
    if mat_data is None:
        return
    
    main_key = None
    for key in mat_data.keys():
        if not key.startswith('__') and isinstance(mat_data[key], np.ndarray):
            main_key = key
            break
    
    if main_key:
        print(f"Ana veri anahtarı: {main_key}")
        battery_struct = mat_data[main_key]
        print(f"Veri tipi: {type(battery_struct)}")
        print(f"Veri şekli: {battery_struct.shape}")
        
        if hasattr(battery_struct, 'dtype') and battery_struct.dtype.names:
            print("Veri alanları:", battery_struct.dtype.names)
            
            for field_name in battery_struct.dtype.names:
                field_data = battery_struct[field_name]
                field_data = safe_extract(field_data)
                
                print(f"\n{field_name}:")
                print(f"  - Tip: {type(field_data)}")
                if hasattr(field_data, 'shape'):
                    print(f"  - Şekil: {field_data.shape}")
    
    return main_key

if 'battery_data' in locals():
    main_key = explore_battery_structure(battery_data)


Ana veri anahtarı: B0005
Veri tipi: <class 'numpy.ndarray'>
Veri şekli: (1, 1)
Veri alanları: ('cycle',)

cycle:
  - Tip: <class 'numpy.ndarray'>
  - Şekil: (1, 616)


In [11]:
# Hücre 3: Veri yapısını keşfedin
def explore_battery_structure(mat_data):
    """
    Batarya verisi yapısını detaylı inceler
    """
    if mat_data is None:
        return
    
    # Ana veri anahtarını bul
    main_key = None
    for key in mat_data.keys():
        if not key.startswith('__') and isinstance(mat_data[key], np.ndarray):
            main_key = key
            break
    
    if main_key:
        print(f"Ana veri anahtarı: {main_key}")
        battery_struct = mat_data[main_key]
        print(f"Veri tipi: {type(battery_struct)}")
        print(f"Veri şekli: {battery_struct.shape}")
        
        # Eğer structured array ise alanları listele
        if hasattr(battery_struct, 'dtype') and battery_struct.dtype.names:
            print("Veri alanları:", battery_struct.dtype.names)
            
            # Her alanın içeriğini güvenli bir şekilde al
            for field_name in battery_struct.dtype.names:
                field_data = battery_struct[field_name]
                
                # Array boyutuna göre doğru indeksleme
                if field_data.ndim == 2:
                    field_data = field_data[0, 0]
                elif field_data.ndim == 1:
                    field_data = field_data[0]
                
                print(f"\n{field_name}:")
                print(f"  - Tip: {type(field_data)}")
                if hasattr(field_data, 'shape'):
                    print(f"  - Şekil: {field_data.shape}")
    
    return main_key

# Veri yapısını keşfedin
if 'battery_data' in locals():
    main_key = explore_battery_structure(battery_data)


Ana veri anahtarı: B0005
Veri tipi: <class 'numpy.ndarray'>
Veri şekli: (1, 1)
Veri alanları: ('cycle',)

cycle:
  - Tip: <class 'numpy.ndarray'>
  - Şekil: (1, 616)
